#### Variable Description cheat sheet

Gender: None

Age: None

Height: None

Weight: None

family_history_with_overweight: Has a family member suffered or suffers from overweight?

FAVC: Do you eat high caloric food frequently?

FCVC: Do you usually eat vegetables in your meals?

NCP: How many main meals do you have daily?

CAEC: Do you eat any food between meals?

SMOKE: Do you smoke?

CH2O: How much water do you drink daily?

SCC: Do you monitor the calories you eat daily?

FAF: How often do you have physical activity?

TUE: How much time do you use technological devices such as cell phone, videogames, television computer and others?"

CALC: How often do you drink alcohol?

MTRANS: Which transportation do you usually use?

NObeyesdad: Obesity level



#### Load The data

In [1]:
import pandas as pd
# df = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")


In [2]:
from uci_data import variables, df
df["BMI"] = df["Weight"] / df["Height"]

continuous_vars = variables[(variables["type"] == "Continuous") |
                            (variables["type"] == "Integer")]["name"].to_list() + ["BMI"]


categorical_vars = variables[(variables["type"] == "Categorical") |
                            (variables["type"] == "Binary")]["name"].to_list()


#A few of the NCP values (number of meals per day) are long floats which doesnt make sence
#I rounded them and will treat as categorical vars since there are only 4 values
df["NCP"] = df["NCP"].astype(int).round()
continuous_vars.remove("NCP")
categorical_vars.append("NCP")

variables[["name","description"]]


,name,description
0,Gender,None
1,Age,None
2,Height,None
3,Weight,None
4,family_history_with_overweight,Has a family member suffered or suffers from o...
5,FAVC,Do you eat high caloric food frequently?
6,FCVC,Do you usually eat vegetables in your meals?
7,NCP,How many main meals do you have daily?
8,CAEC,Do you eat any food between meals?
9,SMOKE,Do you smoke?


#### Describe the continuous variables distribution

In [3]:
df[continuous_vars].describe()

,Age,Height,Weight,FCVC,CH2O,FAF,TUE,BMI
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.008011,1.010298,0.657866,50.638549
std,6.345968,0.093305,26.191172,0.533927,0.612953,0.850592,0.608927,14.265599
min,14.000000,1.450000,39.000000,1.000000,1.000000,0.000000,0.000000,23.788477
25%,19.947192,1.630000,65.473343,2.000000,1.584812,0.124505,0.000000,40.469309
50%,22.777890,1.700499,83.000000,2.385502,2.000000,1.000000,0.625350,49.376062
75%,26.000000,1.768464,107.430682,3.000000,2.477420,1.666678,1.000000,63.496039
max,61.000000,1.980000,173.000000,3.000000,3.000000,3.000000,2.000000,92.513369


#### Count the frequency of categorical varaibles

In [4]:
from pprint import pprint
cts = df.value_counts()
count_dict = {}
for v in categorical_vars:
    count_dict[v] = df[v].value_counts().to_dict() 

pprint(count_dict)

{'CAEC': {'Always': 53, 'Frequently': 242, 'Sometimes': 1765, 'no': 51},
 'CALC': {'Always': 1, 'Frequently': 70, 'Sometimes': 1401, 'no': 639},
 'FAVC': {'no': 245, 'yes': 1866},
 'Gender': {'Female': 1043, 'Male': 1068},
 'MTRANS': {'Automobile': 457,
            'Bike': 7,
            'Motorbike': 11,
            'Public_Transportation': 1580,
            'Walking': 56},
 'NCP': {1: 395, 2: 285, 3: 1362, 4: 69},
 'NObeyesdad': {'Insufficient_Weight': 272,
                'Normal_Weight': 287,
                'Obesity_Type_I': 351,
                'Obesity_Type_II': 297,
                'Obesity_Type_III': 324,
                'Overweight_Level_I': 290,
                'Overweight_Level_II': 290},
 'SCC': {'no': 2015, 'yes': 96},
 'SMOKE': {'no': 2067, 'yes': 44},
 'family_history_with_overweight': {'no': 385, 'yes': 1726}}


#### continuous scatter plots

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import sean_resources as sr

# First plot: Continuous Data Correlations
path = "Plots/CorrelationX1.png"
main = "Weight Gain Continuous Data Correlations"

# Create the pair plot
g = sns.pairplot(df[continuous_vars])
g.map_upper(sr.corrfunc)
plt.suptitle(main, y=1, fontsize=16)
plt.savefig(path)
plt.show()


KeyboardInterrupt



Error in callback <function _draw_all_if_interactive at 0x0000021808080180> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



Error in callback <function flush_figures at 0x000002182232AA20> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



#### Continuous correlations

Correlations are a little hard to read in the previous code so I made a sepearte plot of numeric correlations.

In [ ]:
corr_matrix = df[continuous_vars].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.savefig("Plots/CorrelationNumericX1.png")
plt.show()

#### Categorical correlations

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import math

# Create pairs of categorical variables
pairs = list(itertools.combinations(categorical_vars, 2))

# Calculate the number of rows and columns needed for the grid
num_pairs = len(pairs)
cols = int(math.ceil(math.sqrt(num_pairs)))
rows = int(math.ceil(num_pairs / cols))

# Set up the matplotlib figure with adjusted size
fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows), sharex=False, sharey=False)

# Flatten axes array for easy iteration
axes = axes.flatten()

# Iterate over pairs and plot
for ax, (var1, var2) in zip(axes, pairs):
    sns.countplot(data=df, x=var1, hue=var2, ax=ax)
    ax.set_title(f'{var1} vs {var2}')
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.05))

# Remove any unused subplots
for i in range(len(pairs), len(axes)):
    fig.delaxes(axes[i])

# Add a title for the whole figure
fig.suptitle("Frequency of Categorical Variables Comparison", fontsize=16)
plt.subplots_adjust(top=0.95, hspace=0.4, wspace=0.4)
plt.savefig("Plots/CorrelationX2.png")

plt.show()

#### Interactions

In [ ]:
df['MTRANS'] = df['MTRANS'].astype('category')

# Create a box plot
plt.figure(figsize=(12, 8))
sns.boxplot(data=df, x='MTRANS', y='FAF')

# Add title and labels
plt.title('Box Plot of FAF by MTRANS')
plt.xlabel('Mode of Transportation (MTRANS)')
plt.ylabel('FAF (Continuous Variable)')
plt.savefig("Plots/Box Plot of FAF by MTRANS.png")

#### Create the model

In [ ]:
from scipy import stats
from statsmodels.formula.api import ols
model = ols("BMI~(Gender + Age + FAVC + FCVC + CALC + SCC + FAF + CH2O + CAEC+ TUE)",data=df).fit()
m1 = model
#yes: Gender, Age, FAVC, FCVC, CALC, SCC, FAF, CAEC, TUE

vars_used = ["Gender", "Age", "FAVC", "FCVC", "CALC", "SCC", "FAF", "CAEC", "TUE"]
#no: Height, Weight, NObeyesdad, family_history_with_overweight, smoke, NCP, MTRANS


#### Hypothesis testing

In [ ]:
stats.skew(m1.resid)

In [ ]:
stats.kurtosis(m1.resid)

In [ ]:
from statsmodels.stats.anova import anova_lm
anova_lm(m1)

In [ ]:


summary_df = pd.DataFrame({
    'Coefficient': model.params,
    't-value': model.tvalues,
    'p-value': model.pvalues,
})

# summary_df
m1.summary()

In [ ]:
import class_resources as cr

cr.qq_plot(m1, "Plots/QQPlot.png")

#### Check Residuals variance

In [ ]:
import matplotlib.pyplot as plt

df_plot = pd.DataFrame({"Predictions": m1.fittedvalues,
                        "Residuals": m1.resid})

plt.figure(figsize=(10,6))
sns.scatterplot(data=df_plot, x="Predictions", y="Residuals")
plt.axhline(0, linestyle='--', color='red')
plt.title('Assumptions: Residuals vs. Predicted', fontsize=14, ha='center')
plt.xlabel('Predictions')
plt.ylabel('Residuals')

# Save the plot
plt.savefig('Plots/ResVsPred.png')
plt.show()


In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
bp_test = het_breuschpagan(m1.resid, model.model.exog)
bp_test_statistic = bp_test[0]
bp_test_p_value = bp_test[1]
bp_test_f_value = bp_test[2]
bp_test_f_p_value = bp_test[3]

print("bp_test_statistic: ", bp_test_statistic,
        "\nbp_test_p_value: ", bp_test_p_value,
        "\nbp_test_f_value: ", bp_test_f_value,
        "\nbp_test_f_p_value: ", bp_test_f_p_value)


#### Levene

In [ ]:
from scipy.stats import levene

stat, p_value = levene(*[group["BMI"].values for name, group in df.groupby("FAF")])

print(f"Levene's Test Statistic: {stat}")
print(f"Levene's Test p-value: {p_value}")

# Create a box plot
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='FAF', y='BMI')

# Add title and labels
plt.title('Box Plot of Weight by Cell')
plt.xlabel('Cell')
plt.ylabel('Weight')

# Save the plot
plt.savefig("Plots/HIVBP.png")

# Show the plot
plt.show()

In [ ]:
for i in vars_used:
    stat, p_value = levene(*[group["BMI"].values for name, group in df.groupby(i)])
    print(f"Levene's Test for: {i}")
    print(f"Levene's Test Statistic: {stat}")
    print(f"Levene's Test p-value: {p_value}")
    print("\n")


In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# Ljung-Box test
ljung_box_result = acorr_ljungbox(m1.resid, lags=[1], return_df=True)
print(ljung_box_result)

In [ ]:
import os #TODO

# Simulate residuals (residuals from the fitted model)
residuals = model.resid

# Plot residuals vs fitted values
ptype = "ResVpred"
ver = "version"  # Define your version variable
part = "part"    # Define your part variable
mdl = "model"    # Define your model name
loc = os.path.join("A1M1/Plots", "ResVBigBin.png")

plt.figure(figsize=(10, 6))
sns.residplot(x=model.fittedvalues, y=residuals, lowess=True)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.savefig(loc)


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm

aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

# Perform Tukey's HSD test for pairwise comparisons
tukey = pairwise_tukeyhsd(endog=df['BMI'],
                          groups=df['FAVC'].astype(str) + " : " + df['CAEC'].astype(str),
                          alpha=0.05)

# Print Tukey's HSD results
print(tukey)

# Plot and save the Tukey HSD results
plt.figure(figsize=(10, 6))
tukey.plot_simultaneous()
plt.title('Tukey HSD Test for Pairwise Comparisons')
plt.xlabel('Mean Difference')
plt.grid(True)

# Ensure the directory exists
output_dir = "A1M1/Plots"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

plt.savefig(os.path.join(output_dir, "HIVPWC.png"))

In [ ]:
m2 = ols("BMI~(Gender + Age + FAVC + FCVC + CALC + SCC + FAF + CH2O + CAEC + TUE + FAVC*CAEC)", data=df).fit()

# Summary of the model
model_summary = m2.summary()
print(model_summary)

# Save the summary to a LaTeX file
with open("A1M1/Tables/ModelSummaryAll.tex", "w") as f:
    f.write(model_summary.as_latex())

#### Effect Size

In [ ]:
import pingouin as pg

# Calculate effect sizes

# Prepare data for effect size calculation
df['Gender'] = df['Gender'].astype('category').cat.codes
df['FAVC'] = df['FAVC'].astype('category').cat.codes
df['CALC'] = df['CALC'].astype('category').cat.codes
df['SCC'] = df['SCC'].astype('category').cat.codes
df['CAEC'] = df['CAEC'].astype('category').cat.codes

# Create interaction term manually
df['FAVC_CAEC'] = df['FAVC'] * df['CAEC']

# List of predictor variables including interaction term
predictors = ['Gender', 'Age', 'FAVC', 'FCVC', 'CALC', 'SCC', 'FAF', 'CH2O', 'CAEC', 'TUE', 'FAVC_CAEC']

# Calculate effect sizes
effect_sizes = pg.linear_regression(df[predictors], df['BMI'])
print(effect_sizes)

# Save the effect sizes to a LaTeX file
with open("A1M1/Tables/EffectSize.tex", "w") as f:
    f.write(effect_sizes.to_latex())

In [ ]:
# Fit another model (for comparison)
# Compare models using ANOVA
anova_results = sm.stats.anova_lm(m1, m2)
print(anova_results)

# Save the ANOVA comparison to a LaTeX file
with open("A1M1/Tables/ModelCompare.tex", "w") as f:
    f.write(anova_results.to_latex())